## Pytorch Model to classify image. 
- 10 classes. 
- Training done on MNIST data using 2D CNN.
- model performance not tuned; solely for testing AI engineering setup only. 

reference: 
https://github.com/pytorch/examples/blob/master/mnist/main.py

In [63]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
import numpy as np

In [64]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [65]:
# build model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):

        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [66]:
# train model
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % 100 == 0:
          print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
              epoch, batch_idx * len(data), len(train_loader.dataset),
              100. * batch_idx / len(train_loader), loss.item()))

In [67]:
# test model
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [68]:
# dataset
def get_dataset():
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])
    train_data = datasets.MNIST('../data', train=True, download=True,
                       transform=transform)
    test_data = datasets.MNIST('../data', train=False,
                       transform=transform)
    return train_data, test_data

In [69]:
# get train_loader and test_loader
def get_loaders(batch_size, train_data, test_data):
    train_kwargs = {'batch_size': batch_size}
    test_kwargs = {'batch_size': batch_size}

    if torch.cuda.is_available():
        cuda_kwargs = {'num_workers': 1,
                       'pin_memory': True,
                       'shuffle': True}
        train_kwargs.update(cuda_kwargs)
        test_kwargs.update(cuda_kwargs)

    train_loader = torch.utils.data.DataLoader(train_data,**train_kwargs)
    test_loader = torch.utils.data.DataLoader(test_data, **test_kwargs)

    return train_loader, test_loader

In [70]:
# main function
def main(epochs, train_loader, test_loader):
    # Training settings
    torch.manual_seed(1)
    

    model = Net().to(device)
    optimizer = optim.Adadelta(model.parameters(), lr=0.01)

    scheduler = StepLR(optimizer, step_size=1, gamma=0.5)
    for epoch in range(1, epochs + 1):
        train(model, device, train_loader, optimizer, epoch)
        test(model, device, test_loader)
        scheduler.step()

    return model

In [71]:
train_data, test_data = get_dataset()
print(train_data)

Dataset MNIST
    Number of datapoints: 60000
    Root location: ../data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.1307,), std=(0.3081,))
           )


In [72]:
# run main
batch_size = 250
epochs = 20

train_loader, test_loader = get_loaders(batch_size, train_data, test_data)

model = main(epochs, train_loader, test_loader)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.326382
Train Epoch: 1 [25000/60000 (42%)]	Loss: 1.195949
Train Epoch: 1 [50000/60000 (83%)]	Loss: 0.575605

Test set: Average loss: 0.4427, Accuracy: 8879/10000 (89%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.542979
Train Epoch: 2 [25000/60000 (42%)]	Loss: 0.633686
Train Epoch: 2 [50000/60000 (83%)]	Loss: 0.526757

Test set: Average loss: 0.3527, Accuracy: 9064/10000 (91%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.623399
Train Epoch: 3 [25000/60000 (42%)]	Loss: 0.467733
Train Epoch: 3 [50000/60000 (83%)]	Loss: 0.428579

Test set: Average loss: 0.3276, Accuracy: 9097/10000 (91%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.523870
Train Epoch: 4 [25000/60000 (42%)]	Loss: 0.429676
Train Epoch: 4 [50000/60000 (83%)]	Loss: 0.489012

Test set: Average loss: 0.3179, Accuracy: 9134/10000 (91%)

Train Epoch: 5 [0/60000 (0%)]	Loss: 0.454030
Train Epoch: 5 [25000/60000 (42%)]	Loss: 0.448115
Train Epoch: 5 [50000/60000 (83%)]	Loss: 0.491389

Test set: Average loss: 0.3128

In [73]:
# save model (typical weights saving, cannot be use for triton)
path = "model_pytorch_mnist.pt"
torch.save(model.state_dict(), path)

In [74]:
# printing out summary (not useful)
from torchsummary import summary
summary(model, (1, 28, 28))
print(model)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 26, 26]             320
            Conv2d-2           [-1, 64, 24, 24]          18,496
           Dropout-3           [-1, 64, 12, 12]               0
            Linear-4                  [-1, 128]       1,179,776
           Dropout-5                  [-1, 128]               0
            Linear-6                   [-1, 10]           1,290
Total params: 1,199,882
Trainable params: 1,199,882
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.52
Params size (MB): 4.58
Estimated Total Size (MB): 5.10
----------------------------------------------------------------
Net(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (dropout1): Dropout(p=0.25, inplace=Fa

In [75]:
# https://medium.com/nvidia-ai/how-to-deploy-almost-any-hugging-face-model-on-nvidia-triton-inference-server-with-an-8ee7ec0e6fc4
# saving model based on Torchscript way using tracing. This will be used for Triton.

import tensorflow as tf
# load a sample image
example_img, example_label = next(iter(train_loader))
print(example_img.shape)
example_img = example_img.to(device)
model.to(device)

# run the tracing
traced_script_model = torch.jit.trace(model, example_img)

# see model functions
print(traced_script_model.code)

# save the converted model
path_jit = "model_pytorch_mnist_jit.pt"
traced_script_model.save(path_jit)

torch.Size([250, 1, 28, 28])
def forward(self,
    input: Tensor) -> Tensor:
  _0 = self.fc2
  _1 = self.dropout2
  _2 = self.fc1
  _3 = self.dropout1
  _4 = self.conv2
  input0 = torch.relu((self.conv1).forward(input, ))
  input1 = torch.relu((_4).forward(input0, ))
  input2 = torch.max_pool2d(input1, [2, 2], annotate(List[int], []), [0, 0], [1, 1], False)
  input3 = torch.flatten((_3).forward(input2, ), 1, -1)
  input4 = torch.relu((_2).forward(input3, ))
  _5 = (_0).forward((_1).forward(input4, ), )
  return torch.log_softmax(_5, 1, None)



In [76]:
# load model with saved weights and infer
model = Net()
model.load_state_dict(torch.load(path))
model.eval().to(device)
print("label: ", model(example_img))

label:  tensor([[-5.0846e+00, -6.5180e+00, -4.0622e+00,  ..., -7.6214e+00,
         -4.9520e+00, -9.4501e+00],
        [-7.0891e+00, -9.4501e+00, -5.1400e+00,  ..., -6.7569e+00,
         -2.1310e+00, -4.8164e+00],
        [-1.0334e+01, -7.6493e+00, -5.3049e+00,  ..., -1.0309e+01,
         -6.6333e+00, -6.7848e+00],
        ...,
        [-8.7363e+00, -7.6697e+00, -6.6681e+00,  ..., -1.2061e-02,
         -6.0919e+00, -5.2994e+00],
        [-1.3476e+01, -1.9415e+01, -1.5138e+01,  ..., -6.8424e-05,
         -1.2458e+01, -9.9448e+00],
        [-8.4042e+00, -6.5621e-02, -6.8290e+00,  ..., -5.0921e+00,
         -3.0381e+00, -6.0528e+00]], device='cuda:0',
       grad_fn=<LogSoftmaxBackward>)


In [77]:
# infer with traced_script_model
print("label: ", traced_script_model(example_img))

label:  tensor([[-5.0846e+00, -6.5180e+00, -4.0622e+00,  ..., -7.6214e+00,
         -4.9520e+00, -9.4501e+00],
        [-7.0891e+00, -9.4501e+00, -5.1400e+00,  ..., -6.7569e+00,
         -2.1310e+00, -4.8164e+00],
        [-1.0334e+01, -7.6493e+00, -5.3049e+00,  ..., -1.0309e+01,
         -6.6333e+00, -6.7848e+00],
        ...,
        [-8.7363e+00, -7.6697e+00, -6.6681e+00,  ..., -1.2061e-02,
         -6.0919e+00, -5.2994e+00],
        [-1.3476e+01, -1.9415e+01, -1.5138e+01,  ..., -6.8424e-05,
         -1.2458e+01, -9.9448e+00],
        [-8.4042e+00, -6.5621e-02, -6.8290e+00,  ..., -5.0921e+00,
         -3.0381e+00, -6.0528e+00]], device='cuda:0',
       grad_fn=<LogSoftmaxBackward>)


### Infer with own data; remember to upload your own image and change the path accordingly.

In [83]:
from PIL import Image

# pre-processing
def preprocessing():
    INPUT_SHAPE = (28, 28)
    '''
    Return (1, 1, 28, 28) with FP32 input from image
    '''
    img = Image.open('/content/7.png').convert('L')
    img = img.resize(INPUT_SHAPE)
    imgArr = np.asarray(img) / 255
    imgArr = np.expand_dims(imgArr, 0)
    imgArr = np.expand_dims(imgArr, 0)
    imgArr = imgArr.astype(np.float32)
    print(imgArr.shape)
    torch.from_numpy(x).to(device)
    return imgArr

In [89]:
# infer with model loaded from saved weights
model = Net()
model.load_state_dict(torch.load(path))
model.eval().to(device)
print("label: ", model(x))
print(model(x).shape)

label:  tensor([[-3.2908, -4.8340, -3.2013, -2.5351, -4.1441, -3.7628, -4.4674, -0.3773,
         -3.6178, -2.6347]], device='cuda:0', grad_fn=<LogSoftmaxBackward>)
torch.Size([1, 10])


In [91]:
# predict using traced model
prediction = traced_script_model(x)
print("label: ", prediction)
print(prediction.shape)
prediction1 = prediction.reshape(10).cpu()
print(tf.argmax(prediction1.detach().numpy()))

label:  tensor([[-3.2908, -4.8340, -3.2013, -2.5351, -4.1441, -3.7628, -4.4674, -0.3773,
         -3.6178, -2.6347]], device='cuda:0', grad_fn=<LogSoftmaxBackward>)
torch.Size([1, 10])
tf.Tensor(7, shape=(), dtype=int64)


In [82]:
# script for config.pbtxt
##TODO
